# F05. Generate LULC Map

## F05.00. Initialize Earth Engine

In [17]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import ee
import geemap
from gee_utils import get_aoi_from_gaul, get_landsat_composite, add_spectral_indices, split_training_validation, sample_composite, get_aoi_from_gaul_regency, get_training_points_for_aoi

In [18]:
# Initialize Earth Engine
ee.Authenticate()
ee.Initialize()

## User Inputs and Parameters

In [ ]:
# --- Area of Interest (AOI) ---
COUNTRY = "Indonesia"
PROVINCE = "Sumatera Selatan"
REGENCY =  "Banyuasin" 

# --- Time Period ---
START_DATE = '2018-01-01'
END_DATE = '2018-12-31'

# --- Landsat Settings ---
LANDSAT_VERSION = 'LC08'  # Landsat 8 for 2018 data
CLOUD_COVER = 50  

# --- Bands to Use ---
BANDS = [
    'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7',  # All Landsat bands
    'NDVI', 'NBR', 'NDWI', 'EVI2'                          # Key indices for wetland/forest
]

# --- Training Points Configuration ---
USER_TRAINING_POINTS_ASSET = None  # Set to custom asset path if available
BACKUP_TRAINING_POINTS_ASSET = 'projects/ee-rg2icraf/assets/Sumsel_GT_Restore'  

# Training points validation settings
CLASS_PROPERTY = 'kelas'  # Property containing LULC class labels (1-17)
MIN_POINTS_PER_CLASS = 10  # Minimum points per class for quality warnings

# --- Visualization ---
# Color palette for each LULC class (order must match class names)
land_cover_palette = [
    '#006400',  # Undisturbed dry-land forest
    '#228B22',  # Logged-over dry-land forest
    '#4169E1',  # Undisturbed mangrove
    '#87CEEB',  # Logged-over mangrove
    '#2E8B57',  # Undisturbed swamp forest
    '#8FBC8F',  # Logged-over swamp forest
    '#9ACD32',  # Agroforestry
    '#32CD32',  # Plantation forest
    '#8B4513',  # Rubber monoculture
    '#FF8C00',  # Oil palm monoculture
    '#DAA520',  # Other monoculture
    '#ADFF2F',  # Grass/savanna
    '#90EE90',  # Shrub
    '#FFFF00',  # Cropland
    '#FF0000',  # Settlement
    '#D2B48C',  # Cleared land
    '#0000FF'   # Waterbody
]

land_cover_names = [
    'Undisturbed dry-land forest',
    'Logged-over dry-land forest',
    'Undisturbed mangrove',
    'Logged-over mangrove',
    'Undisturbed swamp forest',
    'Logged-over swamp forest',
    'Agroforestry',
    'Plantation forest',
    'Rubber monoculture',
    'Oil palm monoculture',
    'Other monoculture',
    'Grass/savanna',
    'Shrub',
    'Cropland',
    'Settlement',
    'Cleared land',
    'Waterbody'
]

## F05.01. Image Acquisition & Composite Generation

### F05.01.A. Load Area of Interest

In [20]:
# Load AOI boundaries - either regency-level or province-level
if REGENCY is not None:
    print(f"Loading AOI for {REGENCY} Regency, {PROVINCE}, {COUNTRY}...")
    aoi = get_aoi_from_gaul_regency(country=COUNTRY, province=PROVINCE, regency=REGENCY)
    aoi_name = f"{REGENCY} Regency"
    zoom_level = 9  # Regency-level zoom
else:
    print(f"Loading AOI for entire {PROVINCE} Province, {COUNTRY}...")
    aoi = get_aoi_from_gaul(country=COUNTRY, province=PROVINCE)
    aoi_name = f"{PROVINCE} Province"
    zoom_level = 8  # Province-level zoom

# Visualize AOI boundaries
Map = geemap.Map() 
Map.addLayer(aoi, 
            {'color': 'red', 'fillColor': '00000000'}, 
            f'{aoi_name} Boundary')

# Center map with appropriate zoom level
Map.centerObject(aoi, zoom_level)
Map

Loading AOI for entire Sumatera Selatan Province, Indonesia...


Map(center=[-3.2210694545062024, 104.16355582426586], controls=(WidgetControl(options=['position', 'transparen…

### F05.01.B. Generate Composite

In [22]:
landsat_composite = get_landsat_composite(
    aoi=aoi,
    start_date=START_DATE,
    end_date=END_DATE,
    landsat_version=LANDSAT_VERSION,
    cloud_cover=CLOUD_COVER
)

Map = geemap.Map()
Map.centerObject(aoi, 8)

Map.addLayer(landsat_composite, 
            {'bands': ['SR_B4', 'SR_B3', 'SR_B2'], 'min': 0, 'max': 0.3}, 
            'Composite (RGB)')

# Display composite
Map

Map(center=[-3.2210694545062024, 104.16355582426586], controls=(WidgetControl(options=['position', 'transparen…

## F05.02. Generate and Select Covariates

### F05.02.A. Calculate Spectral indices

In [23]:
# Input bands for generate NDVI and NDWI
composite_with_indices = add_spectral_indices(landsat_composite)

# Select NDVI and NDWI bands from the composite
ndvi = composite_with_indices.select('NDVI')
ndwi = composite_with_indices.select('NDWI')

# Add NDVI layer
ndvi_palette = ['#d73027', '#f46d43', '#fdae61', '#fee08b', '#d9ef8b', '#a6d96a', '#66bd63', '#1a9641']

Map = geemap.Map()
Map.centerObject(aoi, 8)

Map.addLayer(ndvi.clip(aoi), 
            {'min': -1, 'max': 1, 'palette': ndvi_palette}, 
            'NDVI')

# Add NDWI layer
ndwi_palette = ['#8B4513', '#DAA520', '#FFFF00', '#ADFF2F', '#00FF00', '#00FFFF', '#0000FF', '#000080']
Map.addLayer(ndwi.clip(aoi), 
            {'min': -1, 'max': 1, 'palette': ndwi_palette}, 
            'NDWI')
# Display map
Map

Map(center=[-3.2210694545062024, 104.16355582426586], controls=(WidgetControl(options=['position', 'transparen…

## F05.03. Define LULC Categories & Reference Data Preparation

### F05.03.A. Prepare training and validation data

In [24]:
# Load and validate training points for BANYUASIN using the new flexible system
print(f"\n=== LOADING TRAINING POINTS ===")
training_points = get_training_points_for_aoi(
    aoi_geometry=aoi,
    user_training_points_asset=USER_TRAINING_POINTS_ASSET,
    backup_training_points_asset=BACKUP_TRAINING_POINTS_ASSET,
    class_property=CLASS_PROPERTY,
    min_points_per_class=MIN_POINTS_PER_CLASS
)

# Split points into training and validation sets
training, validation = split_training_validation(training_points, split=0.7, seed=42)

print(f'\n=== FINAL TRAINING/VALIDATION SPLIT FOR {REGENCY} ===')
print('Training points:', training.size().getInfo())
print('Validation points:', validation.size().getInfo())

# Visualize training and validation points for BANYUASIN
Map = geemap.Map()
Map.centerObject(aoi, 9)  # Regency-level zoom

# Add BANYUASIN boundary for reference
Map.addLayer(aoi, 
            {'color': 'red', 'fillColor': '00000000'}, 
            f'{REGENCY} Boundary')

# Add training and validation points
Map.addLayer(training, 
            {'color': 'blue'}, 
            'Training Points')

Map.addLayer(validation, 
            {'color': 'orange'}, 
            'Validation Points')

# Display points data
Map


=== LOADING TRAINING POINTS FOR None REGENCY ===
[INFO] Using backup training points: projects/ee-rg2icraf/assets/Sumsel_GT_Restore
[SUCCESS] Training points validation passed:
  - Total points: 39003
  - Points in AOI: 39003 (100.0%)
  - Geometry type: Point
  - CRS: EPSG:4326
[SUCCESS] Successfully loaded backup training points

[INFO] Final training points summary:
  - Source: backup asset: projects/ee-rg2icraf/assets/Sumsel_GT_Restore
  - Points in AOI: 39003
[INFO] Class Distribution Analysis:
  - Total classes: 16
  - Total points: 39003
  - Average points per class: 2437.7
  - Class 1: 916 points (2.3%)
  - Class 10: 637 points (1.6%)
  - Class 11: 3775 points (9.7%)
  - Class 12: 4695 points (12.0%)
  - Class 13: 4043 points (10.4%)
  - Class 14: 1632 points (4.2%)
  - Class 15: 1360 points (3.5%)
  - Class 16: 648 points (1.7%)
  - Class 17: 1057 points (2.7%)
  - Class 2: 684 points (1.8%)
  - Class 3: 9467 points (24.3%)
  - Class 4: 1089 points (2.8%)
  - Class 6: 374 poin

Map(center=[-3.2210694545062024, 104.16355582426586], controls=(WidgetControl(options=['position', 'transparen…

## F05.04. Feature Extraction

### F05.04.A. Optimized sampling

In [26]:
# Sample satellite data at training and validation point locations for BANYUASIN
print(f"Sampling satellite data")
training_samples = sample_composite(composite_with_indices, training, BANDS, class_property=CLASS_PROPERTY)
validation_samples = sample_composite(composite_with_indices, validation, BANDS, class_property=CLASS_PROPERTY)


Sampling satellite data


## F05.05. Model Training & Classification

### F05.05.A. Model training using RandomForest

In [ ]:
# Configure Random Forest classifier optimized for BANYUASIN's diverse land cover
print(f"Training Random Forest classifier")
classifier = ee.Classifier.smileRandomForest(
    # Optimized hyper-parameters for BANYUASIN's 17 LULC classes
    numberOfTrees=100,  # Balanced performance vs speed
    variablesPerSplit=3,  # Good for 10 input features
    minLeafPopulation=2,  # Prevent overfitting
    bagFraction=0.7,  # Robust sampling
    seed=42  # Reproducible results
)

# Train the classifier using BANYUASIN training samples
trained = classifier.train(
    features=training_samples,
    classProperty=CLASS_PROPERTY,
    inputProperties=BANDS
)
print(f"Classifier training complete.")

Training Random Forest classifier
Classifier training complete for None.


### F05.05.B. Classification

In [30]:
print(f"Applying classification")
classified = (composite_with_indices.select(BANDS)
              .classify(trained)
              .set('system:time_start', ee.Date(START_DATE).millis())
              .set('regency', REGENCY)
              .set('province', PROVINCE))
print(f"Classification complete")

Applying classification
Classification complete


### F05.05.C. Validation

In [32]:
# Validate classification accuracy for BANYUASIN
print(f"\n=== ACCURACY ASSESSMENT FOR {aoi_name} ===")
validated = validation_samples.classify(trained)
confusion_matrix = validated.errorMatrix(CLASS_PROPERTY, 'classification')

overall_accuracy = confusion_matrix.accuracy().getInfo()
kappa = confusion_matrix.kappa().getInfo()

print(f'Overall Accuracy for {aoi_name}: {overall_accuracy:.4f} ({overall_accuracy*100:.2f}%)')
print(f'Kappa Coefficient for {aoi_name}: {kappa:.4f}')

# Interpretation for BANYUASIN results
if overall_accuracy >= 0.7:
    print(f'✓ Good accuracy achieved for {aoi_name} classification')
elif overall_accuracy >= 0.6:
    print(f'~ Moderate accuracy for {aoi_name} - acceptable for proof of concept')
else:
    print(f'! Lower accuracy for {aoi_name} - consider additional training data or parameter tuning')


=== ACCURACY ASSESSMENT FOR Sumatera Selatan Province ===
Overall Accuracy for Sumatera Selatan Province: 0.5934 (59.34%)
Kappa Coefficient for Sumatera Selatan Province: 0.5328
! Lower accuracy for Sumatera Selatan Province - consider additional training data or parameter tuning


### F05.05.D. Visualisation

In [ ]:
print(f"Creating visualization map ")
Map = geemap.Map()
Map.centerObject(aoi, 9)  


Map.addLayer(aoi, 
            {'color': 'red', 'fillColor': '00000000'}, 
            f'{aoi_name} Boundary')

Map.addLayer(
    landsat_composite,
    {'bands': ['SR_B4', 'SR_B3', 'SR_B2'], 'min': 0, 'max': 0.3},
    f'{REGENCY} RGB Composite (2018)'
)

# Add BANYUASIN land cover classification
Map.addLayer(
    classified.clip(aoi),
    {'min': 1, 'max': 17, 'palette': land_cover_palette},
    f'{aoi_name} Land Cover Classification (2018)'
)

# Create legend for BANYUASIN LULC classes
legend_dict = dict(zip(land_cover_names, land_cover_palette))
Map.add_legend(
    title=f"{aoi_name} Land Cover Classes",
    legend_dict=legend_dict,
    draggable=True
)

# Add layer control and display
Map.addLayerControl()
print(f"Map ready for {aoi_name} LULC classification!")
Map

Creating visualization map 


Map ready for Sumatera Selatan Province  LULC classification!


Map(center=[-3.2210694545062024, 104.16355582426586], controls=(WidgetControl(options=['position', 'transparen…

### F05.05.E. Export result

In [ ]:
def export_lulc_results():
    """Export LULC classification results to Google Drive"""
    
    year = int(START_DATE[:4])
    
    # Export BANYUASIN land cover classification
    print(f"Preparing export for {aoi_name} regency LULC classification...")
    task = ee.batch.Export.image.toDrive(
        image=classified.clip(aoi),
        description=f'LULC_{aoi_name}_{year}_17Classes',
        folder=f'LULC_Classifications_{aoi_name}',
        region=aoi,
        scale=30,  # 30m Landsat resolution
        maxPixels=1e9,
        crs='EPSG:4326',
        fileFormat='GeoTIFF',
        formatOptions={'cloudOptimized': True}
    )
    
    print(f"Export task created for {aoi_name} :")
    print(f"  - File: LULC_{aoi_name}_{year}_17Classes.tif")
    print(f"  - Folder: LULC_Classifications_{aoi_name}")
    print(f"  - Resolution: 30m")
    print(f"  - Classes: 17 LULC types")
    print("\nTo start export, run: task.start()")
    
    return task

# Uncomment the line below to create the export task
# export_task = export_lulc_results()